In [3]:
1st Problem

SyntaxError: invalid syntax (264621872.py, line 1)

In [4]:
from pyspark.sql import SparkSession
from pyspark.sql import SparkSession
from pyspark.sql.functions import split, col
from pyspark.sql.functions import to_date
from pyspark.sql.functions import *


# Create a Spark session
spark = SparkSession.builder.appName("ReadTabSeparatedData").getOrCreate()

# Specify Input file path
file_path1 = "./us_softball_league.tsv"
file_path2 = "./unity_golf_club.csv"

# Read tab-separated and comma separted data using Spark
df = spark.read.csv(file_path1, sep='\t', header=True, inferSchema=True)
df1 = spark.read.csv(file_path2, sep=',', header=True, inferSchema=True)

print("the first df",df.count())
print("the second df1",df1.count())
print("total input",df.count()+df1.count())
# df1.show()
# Display the DataFrame
#df.show()
#df.printSchema()
df_split = df.withColumn("first_name", split(col("name"), " ")[1])\
             .withColumn("last_name", split(col("name"), " ")[0]) \
             .withColumn('date_of_birth',date_format(to_date(col("date_of_birth"),"MM/dd/yyyy"),'yyyy/dd/MM'))\
             .select("first_name", "last_name", *df.columns) \
             .withColumn("us_state", upper(substring(col("us_state"), 1, 2)))\
             .drop("name")

# Display the DataFrame with split columns
print("DataFrame with Split Columns:")
df_split.show()


combined_df = df_split.union(df1)
print("final_df",combined_df.count())
print("combined_df")
combined_df.show()

# Save the combined DataFrame to a master file
output_file_name = "./final_file"
df.write.csv(output_file_name, header=True, mode="overwrite")
# Stop the Spark session
spark.stop()

the first df 7652
the second df1 8466
total input 16118
DataFrame with Split Columns:
+----------+-----------+-------------+----------+-----------+-----+-------------+--------+
|first_name|  last_name|date_of_birth|company_id|last_active|score|joined_league|us_state|
+----------+-----------+-------------+----------+-----------+-----+-------------+--------+
|   Brennan|    Mikayla|   1966/02/11|         2| 07/04/2018|   84|         1989|      IL|
|    Holmes|     Thomas|   1962/29/11|         1| 05/15/2018|   92|         1972|      WI|
|     Jones|      Corey|   1964/20/12|         7| 08/25/2018|   47|         2007|      NE|
|    Howard|      Laura|   1989/26/04|         8| 04/15/2018|   76|         1976|      NE|
|Mclaughlin|     Daniel|   1966/19/06|        13| 05/10/2018|   56|         1986|      RH|
|   Patrick|    Patrick|   1971/09/08|         7| 06/15/2018|   37|         2013|      VE|
|  Browning|     Robert|   1961/02/02|         4| 01/02/2019|   89|         1971|      WI|
|   

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
from sqlalchemy import create_engine

# Initialize Spark session
#spark = SparkSession.builder.appName("DataModelExample").getOrCreate()

df_large_file = spark.read.csv(file_path1, sep=',', header=True, inferSchema=True)



# Write data to a local PostgreSQL database
db_url = "jdbc:postgresql://localhost:5432/your_database_name"
properties = {"user": "your_username", "password": "your_password", "driver": "org.postgresql.Driver"}

df_data.write.jdbc(url=db_url, table="table_data, mode="overwrite", properties=properties)

# Stop the Spark session
spark.stop()

In [5]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import broadcast, col

# Create a Spark session
spark = SparkSession.builder.appName("BroadcastJoinExample").getOrCreate()

# Sample DataFrames
file_path1 = "./us_softball_league.tsv"
file_path2 = "./companies.csv"

# Read tab-separated and comma-separated data using Spark
df_large_file = spark.read.csv(file_path1, sep='\t', header=True, inferSchema=True)
df_small_file = spark.read.csv(file_path2, sep=',', header=True, inferSchema=True)

# Show the small and large DataFrames
# df_small_file.show()
# df_large_file.show()



# Broadcast the small DataFrame
broadcast_mapping_state = broadcast(df_small_file)

# Register the small DataFrame as a temporary SQL table
broadcast_mapping_state.createOrReplaceTempView("company_mapping")

# Register the large DataFrame as a temporary SQL table
df_large_file.createOrReplaceTempView("large_table")

# Define a Spark SQL query to replace company_id with company name
sql_query = """
    SELECT l.name,l.date_of_birth,l.last_active,l.score,l.joined_league,l.us_state, c.name AS company_id
    FROM large_table l
    LEFT JOIN company_mapping c ON l.company_id = c.id
"""

# Execute the Spark SQL query
df_output = spark.sql(sql_query)

# Show the result
df_output.show()

# Stop the Spark session
spark.stop()


+-------------------+-------------+-----------+-----+-------------+------------+--------------------+
|               name|date_of_birth|last_active|score|joined_league|    us_state|          company_id|
+-------------------+-------------+-----------+-----+-------------+------------+--------------------+
|    Mikayla Brennan|   11/02/1966| 07/04/2018|   84|         1989|    Illinois|        Keller Group|
|      Thomas Holmes|   11/29/1962| 05/15/2018|   92|         1972|   Wisconsin|Brown, Vasquez an...|
|        Corey Jones|   12/20/1964| 08/25/2018|   47|         2007|  New Mexico|Alvarez, Schaefer...|
|       Laura Howard|   04/26/1989| 04/15/2018|   76|         1976|  New Jersey|Smith, Torres and...|
|  Daniel Mclaughlin|   06/19/1966| 05/10/2018|   56|         1986|Rhode Island|Mullen, Huffman a...|
|    Patrick Patrick|   08/09/1971| 06/15/2018|   37|         2013|     Vermont|Alvarez, Schaefer...|
|    Robert Browning|   02/02/1961| 01/02/2019|   89|         1971|   Wisconsin|  

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, to_date
from pyspark.sql.types import BooleanType

# Create a Spark session
#spark = SparkSession.builder.appName("DataCleaningExample").getOrCreate()

# Sample DataFrame
file_path = "./us_softball_league.tsv"
df = spark.read.csv(file_path, sep='\t', header=True, inferSchema=True)

# Show the original DataFrame
df.show()

# Define the criteria for impossible chronological combinations
def is_bad_record(date_of_birth, last_active):
    # Add your specific criteria for impossible chronological combinations
    return to_date(date_of_birth, 'MM/dd/yyyy') > to_date(last_active, 'MM/dd/yyyy')

# Register the UDF
is_bad_record_udf = spark.udf.register("is_suspect_record", is_suspect_record, BooleanType())

# Filter suspect records and write them to a separate file
suspect_records = df.filter(is_bad_record_udf(col("date_of_birth"), col("last_active")))
suspect_records.write.mode("overwrite").csv("./suspect_records.csv", header=True, sep='\t')

# Drop suspect records from the main DataFrame
cleaned_df = df.filter(~is_suspect_record_udf(col("date_of_birth"), col("last_active")))

# Show the cleaned DataFrame
cleaned_df.show()

# Stop the Spark session
spark.stop()

In [ ]:
Part 3: Follow-up
    
Readme.md
1.Remove any redundant or unnecessary code breaking it into modularity.
2.Use some Environemtn kind of varibale.
3.Perform Unit testing
4.Making the diffrenent Depolyment Environment
5.Use some orchestartion tool
6.Enhance code documentation for better understanding.
7.USE DRY PRINCIPLE